<a href="https://colab.research.google.com/github/rahiakela/data-learning-research-and-practice/blob/main/deep-learning-with-python-by-francois-chollet/2-mathematical-building-blocks/2_neural_network_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Neural Network from scratch

You should now have a general understanding
of what’s going on behind the scenes in a neural network. What was a magical
black box at the start, has turned into a clearer picture.

<img src='https://github.com/rahiakela/data-learning-research-and-practice/blob/main/deep-learning-with-python-by-francois-chollet/2-mathematical-building-blocks/images/2.png?raw=1' width='800'/>

- the model, composed of layers that are chained together, maps the input data to predictions. 
- The loss function then compares these predictions
to the targets, producing a loss value: a measure of how well the model’s predictions match what was expected. 
- The optimizer uses this loss value to update the model’s weights.



##Setup

In [12]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
import math

##NN using Keras

Now you understand that the input images are stored in NumPy tensors, which are
here formatted as float32 tensors of shape (60000, 784) (training data) and (10000,
784) (test data) respectively.

In [2]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32") / 255

11501568/11490434 [==============================] - 0s 0us/step


Now you understand that this model consists of a chain of two Dense layers, that each
layer applies a few simple tensor operations to the input data, and that these operations
involve weight tensors. Weight tensors, which are attributes of the layers, are
where the knowledge of the model persists.

In [3]:
model = keras.Sequential([
   layers.Dense(512, activation="relu"),
   layers.Dense(10, activation="softmax")                      
])

Now you understand that sparse_categorical_crossentropy is the loss function
that’s used as a feedback signal for learning the weight tensors, and which the training
phase will attempt to minimize. 

You also know that this reduction of the loss
happens via mini-batch stochastic gradient descent. The exact rules governing a specific
use of gradient descent are defined by the rmsprop optimizer passed as the first
argument.

In [4]:
model.compile(optimizer="rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

Now you understand what happens when you call fit: the model will start to iterate
on the training data in mini-batches of 128 samples, 5 times over (each iteration over
all the training data is called an epoch).

For each batch, the model will compute the
gradient of the loss with regard to the weights (using the Backpropagation algorithm,
which derives from the chain rule in calculus) and move the weights in the direction
that will reduce the value of the loss for this batch.

In [5]:
model.fit(train_images, train_labels, epochs=5, batch_size=128)

Epoch 1/5
469/469 [==============================] - 5s 9ms/step - loss: 0.2560 - accuracy: 0.9260
Epoch 2/5
469/469 [==============================] - 4s 9ms/step - loss: 0.1036 - accuracy: 0.9694
Epoch 3/5
469/469 [==============================] - 4s 9ms/step - loss: 0.0680 - accuracy: 0.9797
Epoch 4/5
469/469 [==============================] - 4s 9ms/step - loss: 0.0499 - accuracy: 0.9852
Epoch 5/5
469/469 [==============================] - 4s 9ms/step - loss: 0.0383 - accuracy: 0.9886


After these 5 epochs, the model will have performed 2,345 gradient updates (469
per epoch), and the loss of the model will be sufficiently low that the model will be
capable of classifying handwritten digits with high accuracy.

At this point, you already know most of what there is to know about neural networks.
Let’s prove it by reimplementing a simplified version of that first example
“from scratch” in TensorFlow, step by step.

##NN from scratch in TensorFlow

What better demonstrates full, unambiguous understanding than implementing everything
from scratch? Of course, what “from scratch” means here is relative: we won’t
reimplement basic tensor operations, and we won’t implement backpropagation. But
we’ll go to such a low level that we will barely use any Keras functionality at all.

###A simple Dense class

You’ve learned earlier that the Dense layer implements the following input transformation,
where W and b are model parameters, and activation is an element-wise
function (usually relu, but it would be softmax for the last layer):

```python
output = activation(dot(W, input) + b)
```

Let’s implement a simple Python class, NaiveDense, that creates two TensorFlow
variables, W and b, and exposes a `__call__()` method that applies the preceding
transformation.

In [7]:
class NaiveDense:

  def __init__(self, input_size, output_size, activation):
    self.activation = activation

    # Create a matrix, W, of shape (input_size, output_size), initialized with random values
    w_shape = (input_size, output_size)
    w_initial_value = tf.random.uniform(w_shape, minval=0, maxval=1e-1)
    self.W = tf.Variable(w_initial_value)

    # Create a vector, b, of shape (output_size,), initialized with zeros
    b_shape = (output_size, )
    b_initial_value = tf.zeros(b_shape)
    self.b = tf.Variable(b_initial_value)

  def __call__(self, inputs):
    return self.activation(tf.matmul(inputs, self.W) + self.b)

  # Convenience method for retrieving the layer’s weights
  @property
  def weights(self):
    return [self.W, self.b]

###A simple Sequential class

Now, let’s create a NaiveSequential class to chain these layers. It wraps a list of layers
and exposes a `__call__()` method that simply calls the underlying layers on the
inputs, in order. 

It also features a weights property to easily keep track of the layers’
parameters.

In [9]:
class NaiveSequential:

  def __init__(self, layers):
    self.layers = layers

  def __call__(self, inputs):
    x = inputs
    for layer in self.layers:
      x = layer(x)
    return x

  @property
  def weights(self):
    weights = []
    for layer in self.layers:
      weights += layer.weights
    return weights

Using this NaiveDense class and this NaiveSequential class, we can create a mock
Keras model:

In [10]:
model = NaiveSequential([
    NaiveDense(input_size=28 * 28, output_size=512, activation=tf.nn.relu),
    NaiveDense(input_size=512, output_size=10, activation=tf.nn.softmax)                     
])

assert len(model.weights) == 4

###A batch generator

Next, we need a way to iterate over the MNIST data in mini-batches. This is easy:

In [13]:
class BatchGenerator:
  def __init__(self, images, labels, batch_size=128):
    assert len(images) == len(labels)

    self.index = 0
    self.images = images
    self.labels = labels
    self.batch_size = batch_size
    self.num_batches = math.ceil(len(images) / batch_size)

  def next(self):
    images = self.images[self.index: self.index + self.batch_size]
    labels = self.labels[self.index: self.index + self.batch_size]
    self.index += self.batch_size
    return images, labels

##Running one training step